# LOOCV for Evaluating Machine Learning Algorithms

by Jason Brownlee on July 27, 2020.[Here](https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/) in [Python Machine Learning](https://machinelearningmastery.com/category/python-machine-learning/)

The Leave-One-Out Cross-Validation, or LOOCV, procedure is used to estimate the performance of machine learning algorithms when they are __used to make predictions on data not used to train the model__.

After completing this tutorial, you will know:

- The leave-one-out cross-validation procedure is appropriate when you have a small dataset or when an accurate estimate of model performance is more important than the computational cost of the method.
- How to use the scikit-learn machine learning library to perform the leave-one-out cross-validation procedure.
- How to evaluate machine learning algorithms for classification and regression using leave-one-out cross-validation.

## Tutorial Overview
This tutorial is divided into three parts; they are:

1. LOOCV Model Evaluation
2. LOOCV Procedure in Scikit-Learn
    - 2.1 Compare cross_val_score
3. LOOCV to Evaluate Machine Learning Models
    - 3.1 LOOCV for Classification
    - 3.2 LOOCV for Regression

## 1. LOOCV Model Evaluation
Cross-validation, or k-fold cross-validation, is a procedure used to estimate the performance of a machine learning algorithm when making predictions on data not used during the training of the model.

The cross-validation has a single hyperparameter “k” that controls the number of subsets that a dataset is split into. Once split, __each subset is given the opportunity to be used as a test set while all other subsets together are used as a training dataset__.

__Leave-one-out cross-validation|__, or LOOCV, is a configuration of k-fold cross-validation where k is set to the number of examples in the dataset.

LOOCV is an extreme version of k-fold cross-validation that has the maximum computational cost. It requires one model to be created and evaluated for each example in the training dataset.

__The benefit of so many fit and evaluated models is a more robust estimate of model performance as each row of data is given an opportunity to represent the entirety of the test dataset__.

Given the computational cost, LOOCV is __not appropriate for very large datasets such as more than tens or hundreds of thousands of examples__, or for models that are costly to fit, such as neural networks.

- __Don’t Use LOOCV__: Large datasets or costly models to fit.

Further, __given that no sampling of the training dataset is used, this estimation procedure is deterministic__, unlike train-test splits and other k-fold cross-validation confirmations that provide a stochastic estimate of model performance.

- __Use LOOCV__: Small datasets or when estimated model performance is critical.

## 2. LOOCV Procedure in Scikit-Learn

[LeaveOneOut class.](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html)

In [1]:
# loocv to manually evaluate the performance of a random forest classifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# create dataset
X, y = make_blobs(n_samples=100, random_state=1)

The method has no configuration, therefore, no arguments are provided to create an instance of the class.

In [2]:
# create loocv procedure
cv = LeaveOneOut()

Once created, the split() function can be called and provided the dataset to enumerate.

Each iteration will return the row indices that can be used for the train and test sets from the provided dataset.

These indices can be used on the input (X) and output (y) columns of the dataset array to split the dataset.

In [3]:
# enumerate splits
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]

    # fit model
    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, y_train)
    
    # evaluate model
    yhat = model.predict(X_test)
    
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])

# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print('Accuracy: %.3f' % acc)

Accuracy: 0.990


Given that the dataset has 100 examples, it means that 100 train/test splits of the dataset were created, with each single row of the dataset given an opportunity to be used as the test set. Similarly, 100 models are created and evaluated.

The classification accuracy across all predictions is then reported, in this case as 99 percent.

### 2.1 Compare cross_val_score

The example below demonstrates evaluating the RandomForestClassifier using LOOCV on the same synthetic dataset using the cross_val_score() function.

In [4]:
# loocv to automatically evaluate the performance of a random forest classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# create dataset
X, y = make_blobs(n_samples=100, random_state=1)

# create loocv procedure
cv = LeaveOneOut()

# create model
model = RandomForestClassifier(random_state=1)

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.990 (0.099)


## 3. LOOCV to Evaluate Machine Learning Models

### 3.1 LOOCV for Classification
We will demonstrate how to use LOOCV to evaluate a random forest algorithm on the sonar dataset.

The dataset involves predicting whether sonar returns indicate a rock or simulated mine.

- Sonar Dataset ([sonar.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv))
- Sonar Dataset Description ([sonar.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.names))

In [5]:
# loocv evaluate random forest on the sonar dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [6]:
# load dataset
#url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'
url = '..//..//data//sonar.data.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values

# split into inputs and outputs
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(208, 60) (208,)


In [7]:
# create loocv procedure
cv = LeaveOneOut()

# create model
model = RandomForestClassifier(random_state=1)

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.822 (0.382)


The model is then evaluated using LOOCV and the estimated performance when making predictions on new data has an accuracy of about 82.2 percent.

### 3.2 LOOCV for Regression
We will demonstrate how to use LOOCV to evaluate a random forest algorithm on the housing dataset.

The dataset involves predicting the house price given details of the house’s suburb in the American city of Boston.

- Housing Dataset ([housing.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv))
- Housing Description ([housing.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.names))

In [8]:
# loocv evaluate random forest on the housing dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [9]:
# load dataset
#url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
url = '..//..//data//housing.data.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values

# split into inputs and outputs
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(506, 13) (506,)


In [10]:
# create loocv procedure
cv = LeaveOneOut()

# create model
model = RandomForestRegressor(random_state=1)

# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force positive
scores = absolute(scores)

# report performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 2.182 (2.338)


Use the mean absolute error (MAE) performance metric appropriate for regression.

The model is evaluated using LOOCV and the performance of the model when making predictions on new data is a mean absolute error of about 2.180 (thousands of dollars).